In [1]:
from bs4 import BeautifulSoup
import requests
import re

## Proof of Concept
OC 1, 9 seifim

In [2]:
def simple_parsing(url):
    r  = requests.get(url)
    data = r.text
    return BeautifulSoup(data, "html.parser")

In [25]:
url = 'https://he.wikisource.org/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%90%D7%95%D7%A8%D7%97_%D7%97%D7%99%D7%99%D7%9D_%D7%90'
soup = simple_parsing(url)

In [26]:
span = soup.findAll("span", id=re.compile('סעיף.*'))
len(span)

9

In [3]:
import pandas as pd
import numpy as np

## Scaling

In [4]:
OC_url = 'https://he.wikisource.org/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%90%D7%95%D7%A8%D7%97_%D7%97%D7%99%D7%99%D7%9D'
YD_url = 'https://he.wikisource.org/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%99%D7%95%D7%A8%D7%94_%D7%93%D7%A2%D7%94'
EH_url = 'https://he.wikisource.org/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%90%D7%91%D7%9F_%D7%94%D7%A2%D7%96%D7%A8'
CM_url = 'https://he.wikisource.org/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%97%D7%95%D7%A9%D7%9F_%D7%9E%D7%A9%D7%A4%D7%98'

In [5]:
soup = simple_parsing(OC_url)

In [6]:
simanim = soup.findAll('p')[14]

In [18]:
df = pd.DataFrame([i.text for i in soup.findAll('a', text=simanim)])
df = df[0].str.split('•', expand=True)
df['Section'] = 'OC'
df = df.rename(columns={0:'Number', 1:'Description', 2:'Length'})
df.head()

,Number,Description,Length,Section
0,,None,None,OC
1,סימן א',דין השכמת הבוקר,ובו ט' סעיפים,OC
2,סימן ב',דין לבישת בגדים,ובו ו' סעיפים,OC
3,סימן ג',דין הנהגת בית הכסא,"ובו י""ז סעיפים",OC
4,סימן ד',דיני נטילת ידים,"ובו כ""ג סעיפים",OC


In [19]:
df.tail()

,Number,Description,Length,Section
696,"סימן תרצ""ו",דיני הספד ותענית ועשיית מלאכה בפורים,ובו ח' סעיפים,OC
697,"סימן תרצ""ז",אין אומרים תחינה בארבעה עשר וחמישה עשר שבאדר ...,ובו סעיף אחד,OC
698,,None,None,OC
699,,None,None,OC
700,,None,None,OC


In [20]:
df = df.drop(index=[0, 698, 699, 700])

Strip leading/trailing whitespace

In [21]:
for i in list(df):
    df[i] = df[i].str.strip()

In [22]:
df.Length.unique()

array(["ובו ט' סעיפים", "ובו ו' סעיפים", 'ובו י"ז סעיפים',
       'ובו כ"ג סעיפים', 'ובו סעיף אחד', "ובו ד' סעיפים",
       'ובו י"ב סעיפים', 'ובו ט"ו סעיפים', "ובו ג' סעיפים",
       "ובו ה' סעיפים", "ובו ב' סעיפים", 'ובו י"ג סעיפים',
       'ובו י"א סעיפים', 'ובו נ"ב סעיפים', "ובו י' סעיפים",
       "ובו ח' סעיפים", 'ובו י"ד סעיפים', 'ובו כ"ו סעיפים',
       'ובו כ"ב סעיפים', "ובו ז' סעיפים", 'ובו כ"ז סעיפים',
       'ובו מ"ה סעיפים', "ובו כ' סעיפים", 'ובו י"ח סעיפים',
       'ובו נ"א סעיפים', 'ובו י"ט סעיפים', 'ובו ט"ז סעיפים',
       'ובו מ"ט סעיפים', "ובו ה' הסעיפים", "ובו ב' בסעיפים",
       'ובו ל"ו סעיפים', 'ובו כ"ד סעיפים', 'ובו כ"ה סעיפים'], dtype=object)

In [14]:
df['Seifim'] = df.Length  # backup
swap=dict(zip(df.Length.unique(),[9,17,17,23,1,4,12,15,3,5,2,13,11,52,10,8,14,26,22,7,27,45,20,18,51,19,16,49,5,2,36,24,25]))
df.Length = df.Length.map(swap)

In [16]:
df.head()

,Number,Description,Length,Section,Seifim
1,סימן א',דין השכמת הבוקר,9,OC,ובו ט' סעיפים
2,סימן ב',דין לבישת בגדים,17,OC,ובו ו' סעיפים
3,סימן ג',דין הנהגת בית הכסא,17,OC,"ובו י""ז סעיפים"
4,סימן ד',דיני נטילת ידים,23,OC,"ובו כ""ג סעיפים"
5,סימן ה',דין כוונת הברכות,1,OC,ובו סעיף אחד


In [17]:
df.Length.sum()

4549

In [71]:
soup = simple_parsing(YD_url)

In [74]:
simanim = soup.findAll('p')[13]

In [75]:
for i in soup.findAll('a', text=simanim):
    print (i.text)


סימן א' • מי הם הכשרים לשחוט • ובו י"ד סעיפים
סימן ב' • אם שחיטת עובד כוכבים ומומר כשרה • ובו י"א סעיפים
סימן ג' • שחיטה אינה צריכה כוונה • ובו סעיף אחד
סימן ד' • השוחט לשם עבודת כוכבים או לשם דבר אחר מה דינו • ובו ז' סעיפים
סימן ה' • השוחט לשם קדשים מה דינו • ובו ג' סעיפים
סימן ו' • במה שוחטין • ובו ד' סעיפים
סימן ז' • הקובע סכין בגלגל אם מותר לשחוט בו • ובו סעיף אחד
סימן ח' • שיעור סכין של שחיטה • ובו סעיף אחד
סימן ט' • השוחט בסכין מלובנת • ובו סעיף אחד
סימן י' • יתר דיני סכין • ובו ג' סעיפים
סימן י"א • באיזה זמן שוחטין, ודין השוחט בתוך המים • ובו ד' סעיפים
סימן י"ב • שלא לשחוט בתוך גומא • ובו ב' סעיפים
סימן י"ג • בעלי חיים שאינם צריכים שחיטה • ובו ו' סעיפים
סימן י"ד • דין עובר במעי בהמה • ובו ו' סעיפים
סימן ט"ו • שלא לשחוט בהמה עד יום שמיני ללידתה • ובו ג' סעיפים
סימן ט"ז • דין אותו ואת בנו • ובו י"ב סעיפים
סימן י"ז • דין השוחט בהמה המסוכנת למות • ובו ג' סעיפים
סימן י"ח • דין בדיקת הסכין ופגימותיו • ובו כ' סעיפים
סימן י"ט • דין ברכת השחיטה • ובו ח' סעיפים
סימן כ' • מקום השחיטה בצוו

In [142]:
soup = simple_parsing(EH_url)

In [143]:
soup.findAll('p')[9].text

'א • ב • ג • ד • ה • ו • ז • ח • ט • י • יא • יב • יג • יד • טו • טז • יז • יח • יט • כ • כא • כב • כג • כד • כה • כו • כז • כח • כט • ל • לא • לב • לג • לד • לה • לו • לז • לח • לט • מ • מא • מב • מג • מד • מה • מו • מז • מח • מט • נ • נא • נב • נג • נד • נה • נו • נז • נח • נט • ס • סא • סב • סג • סד • סה • סו • סז • סח • סט • ע • עא • עב • עג • עד • עה • עו • עז • עח • עט • פ • פא • פב • פג • פד • פה • פו • פז • פח • פט • צ • צא • צב • צג • צד • צה • צו • צז • צח • צט • ק • קא • קב • קג • קד • קה • קו • קז • קח • קט • קי • קיא • קיב • קיג • קיד • קטו • קטז • קיז • קיח • קיט • קכ • קכא • קכב • קכג • קכד • קכה • קכו • קכז • קכח • קכט • קל • קלא • קלב • קלג • קלד • קלה • קלו • קלז • קלח • קלט • קמ • קמא • קמב • קמג • קמד • קמה • קמו • קמז • קמח • קמט • קנ • קנא • קנב • קנג • קנד • סדר הגט • קנה • קנו • קנז • קנח • קנט • קס • קסא • קסב • קסג • קסד • קסה • קסו • קסז • קסח • קסט • סדר החליצה • קע • קעא • קעב • קעג • קעד • קעה • קעו • קעז • קעח\n'

In [132]:
simanim = soup.findAll('p')[9]

In [116]:
for i in soup.findAll('a', text=simanim):
    print (i.text)


סימן א • דיני פריה ורביה ושלא לעמוד בלא אשה • ובו י"ד סעיפים
סימן ב • שישתדל כל אדם ליקח לו אשה הגונה • ובו י"א סעיפים
סימן ג • דין בן שהוא ספק לכהונה • ובו ט' סעיפים
סימן ד • מי הם אסורים לבא בקהל ואיזה הנקרא ממזר • ובו ל"ז סעיפים
סימן ה • דין פצוע דכא וכרות שפכה ואיסור סירוס לאדם ולבהמה ולעוף • ובו י"ד סעיפים
סימן ו • נשים האסורות לכהן ודין אשת כהן • ובו ט"ו סעיפים
סימן ז • דין אשה שנשבית ואיזו היא נקראת חללה • ובו כ"ג סעיפים
סימן ח • בכל ספק הולד הולך אחר הזכר • ובו ה' סעיפים
סימן ט • איזו אשה נקראת קטלנית • ובו ב' סעיפים
סימן י • איזו גרושה מותרת לחזור לבעלה • ובו ז' סעיפים
סימן יא • אשה הנחשדת על אשת איש • ובו ח' סעיפים
סימן יב • עדי גט וכן עדים שהעידו מיתת הבעל אם רשאים לישא האשה ההיא • ובו ד' סעיפים
סימן יג • גרושה או אלמנה צריכה להמתין צ' יום טרם שתנשא, ושלא ישא מינקת חבירו ומעוברת חבירו • ובו ? סעיפים
סימן יד • שלא לישא אשה בימי האבל • ובו ? סעיפים
סימן טו • איסור ערוה דאורייתא ודרבנן • ובו ? סעיפים
סימן טז • איסור עובדת כוכבים ושפחה • ובו ? סעיפים
סימן יז • היתר אשת איש שמת 

In [144]:
soup = simple_parsing(CM_url)

In [147]:
new_try = str(soup.findAll('p')[11])

In [149]:
new = BeautifulSoup(new_try, "html.parser")

In [151]:
links = new.findAll('a')

In [119]:
simanim = soup.findAll('p')[11]

In [120]:
type(simanim)

bs4.element.Tag

In [128]:
soup.findAll('a', text=simanim)[11]

IndexError: list index out of range

In [130]:
print (len(soup('a', text=simanim)))

4


In [158]:
for i in links:
    print (i['href'])

/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%97%D7%95%D7%A9%D7%9F_%D7%9E%D7%A9%D7%A4%D7%98_%D7%90
/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%97%D7%95%D7%A9%D7%9F_%D7%9E%D7%A9%D7%A4%D7%98_%D7%91
/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%97%D7%95%D7%A9%D7%9F_%D7%9E%D7%A9%D7%A4%D7%98_%D7%92
/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%97%D7%95%D7%A9%D7%9F_%D7%9E%D7%A9%D7%A4%D7%98_%D7%93
/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%97%D7%95%D7%A9%D7%9F_%D7%9E%D7%A9%D7%A4%D7%98_%D7%94
/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%97%D7%95%D7%A9%D7%9F_%D7%9E%D7%A9%D7%A4%D7%98_%D7%95
/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%97%D7%95%D7%A9%D7%9F_%D7%9E%D7%A9%D7%A4%D7%98_%D7%96
/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A8%D7%95%D7%9A_%D7%97%D7%95%D7%A9%D7%9F_%D7%9E%D7%A9%D7%A4%D7%98_%D7%97
/wiki/%D7%A9%D7%95%D7%9C%D7%97%D7%9F_%D7%A2%D7%A

In [109]:
for i in soup.findAll('a', text=simanim):
    print (i)

<a id="top"></a>
<a class="mw-wiki-logo" href="/wiki/%D7%A2%D7%9E%D7%95%D7%93_%D7%A8%D7%90%D7%A9%D7%99" title="ביקור בעמוד הראשי"></a>
<a href="https://wikimediafoundation.org/"><img alt="Wikimedia Foundation" height="31" src="/static/images/wikimedia-button.png" srcset="/static/images/wikimedia-button-1.5x.png 1.5x, /static/images/wikimedia-button-2x.png 2x" width="88"/></a>
<a href="//www.mediawiki.org/"><img alt="Powered by MediaWiki" height="31" src="/static/images/poweredby_mediawiki_88x31.png" srcset="/static/images/poweredby_mediawiki_132x47.png 1.5x, /static/images/poweredby_mediawiki_176x62.png 2x" width="88"/></a>
